In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

In [3]:
class SimpleNet(nn.Module):
    
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x

In [4]:
train_data_path = "./training_set/"
val_data_path = "./validation_set/"
test_data_path = "./test_set/"

## Transforms.

### Resize(64):
Изображения, на которых будет происходить обучение представлены во многших разрешениях, чтобы повысить производительность обработки, мы масштабируем каждое входящее изображение до разрешения 64 х 64.

### ToTensor():
Превращаем изображения в тензор.

### Normalize:
Нормализуем тензор вокруг определённого набора средних и стандартных точек отклонения.

In [5]:
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ])

In [6]:
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms)
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms)
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms)

In [7]:
batch_size = 64
lr = 0.001
device = 'cpu'
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle = True)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle = True)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [8]:
simplenet = SimpleNet()

In [9]:
optimizer = optim.Adam(simplenet.parameters(), lr=lr)

In [10]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print(f"Epoch: {epoch + 1}, Training Loss: {training_loss}, Validation Loss: {valid_loss}, Accuracy: {num_correct / num_examples}")

In [11]:
train(simplenet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=20, device=device)

C:\Users\chele\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]
C:\Users\chele\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 0, Training Loss: 0.71, Validation Loss: 0.69, accuracy = 0.59


KeyboardInterrupt: 

In [ ]:
labels = ["It's a cat!", "It's a dog!"]

img = Image.open("./test_set/cats/cat.4509.JPG") 
img = img_transforms(img).to(device)


prediction = F.softmax(simplenet(img))
prediction = prediction.argmax()
print(labels[prediction])